In [14]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import yfinance as yf
import scipy as sp
from os import times


##Black Scholes PDE

$$\frac{\partial V}{\partial t}
+ \frac{1}{2}\sigma^{2} S^{2} \frac{\partial^{2} V}{\partial S^{2}}
+ r S \frac{\partial V}{\partial S}
- r V = 0$$

<br>

**Time derivative**

$\frac{\partial V}{\partial t}$ rate of change of option value with respect to time:  $$\frac{V_{i,k} - V_{i,k+1}}{\Delta t}$$

<br>
$\frac{\partial^{2} V}{\partial S^{2}}$ second derivative with respect to stock price:
$$\frac{V_i+1,_k - 2V_i,_k  + V_i-1_,k}{\Delta S^2}$$

<br>


$\frac{\partial V}{\partial S}$ = first derivative with respect to stock price:
$$\frac{V_{i+1,k} - V_{i-1,k}}{2\Delta S}$$

**Explicit FDM**

<br>

  $$V_{i,k} = A_i V_{i-1,k+1} + B_i V_{i,k+1} + C_i V_{i+1,k+1}$$
<br>


where:

$$A_i = \frac{1}{2} \Delta t \left(\sigma^2 i^2 - r i\right)$$

$$B_i = 1 - \Delta t \left(\sigma^2 i^2 + r\right)$$

$$C_i = \frac{1}{2} \Delta t \left(\sigma^2 i^2 + r i\right)$$

The option price today is $V_{i_0, 0}$ where $i_0$ corresponds to the current stock price


**Boundary Conditions for European Calls**:
<br>
$$V(0,t) = 0 {\forall i}$$

$$V(S,T) = \max(S - K,0) $$

$$V(S_{max},t)=S_{max} - Ke^{−r(T−t)}$$
<br>




In [7]:
def number_of_days_asset_level(time_lvl, asset_level):
  return np.zeros((time_lvl + 1,asset_level + 1), dtype=np.float64)

In [4]:
def set_boundary_condition_bottom(MeshGrid):
  for i in range(MeshGrid.shape[0]):
    MeshGrid[i,0] = 0
  return MeshGrid

In [10]:
def set_boundary_condition_top(MeshGrid, timeStep, K, Max):
  for i in range(MeshGrid.shape[1]):
    MeshGrid[Max, i] = max(timeStep * i - K, 0)
  return MeshGrid

In [ ]:
def set_right_boundary_condition(MeshGrid, K, Smax,t, T, r):
  for k in range(MeshGrid.shape[0]):
    MeshGrid[k, -1] = Smax - K * np.exp(-r * (T - t[k]))
  return MeshGrid


In [ ]:

def bse_exp(sigma, r, T, K, N, spatial_step, S0):
    """
    sigma = Volatility
    r = interest rate
    T = Time to maturity in years
    K  = Strike Price
    N = Number of grid points
    spatial_step = 
    """
    start = time.time()
    Smax = max(3 * K, 2.5 * S0)

    M = int(Smax/spatialStep)
    dt = T/N #delta T
    t = np.linspace(0, T, N + 1) #Time axis N+1 points from t=0 to t=T
    S = np.linspace(0, Smax, M + 1) # Stock price axis M+1 points from S=0 S=Smax
    dS = S[1] - S[0]

    # Setting up the boundaries where the value of the option will be V(t_k, S_i)
    Grid = number_of_days_asset_level(N, M)
    Grid = set_boundary_condition_bottom(Grid)
    Grid = set_boundary_condition_top(Grid,spatial_step, K, N)
    Grid = set_right_boundary_condition(Grid, K, Smax, t, T, r )
    #print(Grid)

    A = np.zeros(M+1, dtype=np.float64)
    B = np.zeros(M+1,dtype=np.float64)
    C = np.zeros(M+1, dtype=np.float64)
    #Calculate A_ki B_ki C_ki

    for j in range(1,M):
        A[j] = 0.5 * dt * (sigma ** 2 * j ** 2 - r * j)
        B[j] = 1 - dt * (sigma ** 2 * j ** 2 + r)
        C[j] = 0.5 * dt * (sigma ** 2 * j ** 2 + r * j)
        
    for k in range(N-1, -1, -1):
        for i in range(1, M):
            Grid[k, i] = (
                A[i] * Grid[k+1, i-1] +
                B[i] * Grid[k+1, i] +
                C[i] * Grid[k+1, i+1]
            )
    end = time.time()
    time_taken = end - start
    return Grid, t,S, time_taken